# SPXW Dataset

The spxw weekly dataset contains a snapshop of all the S&P500 Weekly options on the data Sep 3 2025.

In [1]:
from volkit.datasets import spxw

Below is the full dataset.


* ``T`` : float, time to expiry in trading years (``D / 252``)
* ``D`` : int, calendar days to expiry (``ExpDate - Date``)
* ``K`` : float, strike
* ``C_bid``, ``C_ask``, ``C_vol`` : call bid/ask/volume
* ``P_bid``, ``P_ask``, ``P_vol`` : put bid/ask/volume

In [3]:
df = spxw()
df

,T,D,K,C_bid,C_ask,C_vol,P_bid,P_ask,P_vol
0,0.003968,1,2600,3838.60,3855.60,3,0.0,0.05,0
1,0.003968,1,2800,3639.80,3656.10,0,0.0,0.05,0
2,0.003968,1,3000,3438.70,3457.60,0,0.0,0.05,0
3,0.003968,1,3200,3239.90,3256.00,0,0.0,0.05,0
4,0.003968,1,3400,3038.70,3056.30,0,0.0,0.05,0
...,...,...,...,...,...,...,...,...,...
3031,0.341270,86,7410,1.20,1.35,0,890.5,906.50,0
3032,0.341270,86,7420,1.15,1.30,0,900.4,916.90,0
3033,0.341270,86,7430,1.10,1.25,0,910.7,926.70,0
3034,0.341270,86,7500,0.85,1.00,42,980.0,995.70,0


The data loader has two arguments ``min_vol`` and ``D`` that allow you to load and filter subset.

Below we load only the options with a traded volume of at least 100 in both calls and puts, and with 7 days till expiration.

In [11]:
df = spxw(min_vol=100, D=7)
df

,T,D,K,C_bid,C_ask,C_vol,P_bid,P_ask,P_vol
584,0.027778,7,6420,61.8,62.1,106,30.1,30.3,219
585,0.027778,7,6425,58.3,58.6,408,31.6,31.9,273
586,0.027778,7,6430,54.9,55.2,182,33.2,33.5,148
587,0.027778,7,6435,51.6,51.9,172,34.9,35.2,189
588,0.027778,7,6440,48.4,48.7,429,36.7,37.0,374
590,0.027778,7,6450,42.2,42.5,293,40.5,40.8,213
595,0.027778,7,6475,28.4,28.7,289,51.6,52.0,126
